```perl
#!/usr/bin/perl

use warnings;
use strict;
use utf8;

binmode(STDIN, ":utf8");
binmode(STDOUT, ":utf8");

my $title = '';
my $id = '';
while(<>) {
	chomp;
	if(m!<title>([^<]+)</title>!) {
		$title = $1;
	}
        if(m!^      <id>([^<]+)</id>!) {
		$id = $1;
	}
	if(m!{{IPA\|ga!) {
		if($title and $title ne '') {
			print "\n---\n";
			print "$title\n";
			print "ID $id\n";
			$title = '';
			$id = '';
		}
		print "$_\n";
	} elsif(m!Quiggin!) {
		print "$_\n";
	}
}
```

In [38]:
INPUT = "/Users/joregan/Playing/irish-gists/wiktionary/2025-extract.txt"

In [39]:
with open(INPUT) as f:
    text = f.read()
pieces = text.split("\n---\n")

In [40]:
pieces = [p.strip() for p in pieces if p != ""]

In [41]:
pieces = [p for p in pieces if "|ga|" in p]

In [42]:
quiggin = [p for p in pieces if "Quiggin" in p]

In [43]:
sep_quote = [p for p in quiggin if "* {{R:ga:Quiggin" in p]

In [ ]:
sep_quote

In [45]:
import urllib
extracted = []
for p in sep_quote:
    lines = p.split("\n")
    word = lines[0].strip()
    id = lines[1].strip().replace("ID ", "")
    lines = lines[2:]
    if len(lines) == 2:
        raw = lines
    else:
        raw = []
        for line in lines:
            if line.endswith("</text>"):
                line = line.replace("</text>", "").strip()
            if line.startswith("* {{R:ga:Quiggin"):
                raw.append(line)
            elif "Ulster" in line:
                if "<comment>" in line:
                    continue
                raw.append(line)
    data = {}
    data["irish"] = word
    url_word = urllib.parse.quote(word.replace(" ", "_"))
    data["url"] = f"https://en.wiktionary.org/w/index.php?title={url_word}&oldid={id}"
    data["raw"] = list(set(raw))
    extracted.append(data)

In [46]:
import json
with open("quiggin-extract.json", "w") as f:
    json.dump(extracted, f, indent=2)